In [109]:
# Загружаем бибилотеки:
# для работы с массивами данных
import pandas as pd
# для работы со статистикой
from scipy import stats as st
# для работы с регулярными выражениями
import re
# Для работы со стоп-словами
from nltk.corpus import stopwords
# Для работы с текстами
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
# для разделения на выборки
from sklearn.model_selection import train_test_split
# для логистической регрессии
from sklearn.linear_model import LogisticRegression
# Для построения матрицы ошибок
from sklearn.metrics import confusion_matrix

### Задание 1

In [2]:
# Загружаем файл с данными
games = pd.read_csv('vgsales.csv')

print('Как критики относятся к спортивным играм? Предположим, что среднему значению рейтинга до 50 соответсвует посредственное отношение, от 50 до 75 - хорошее, более 75 - превосходное.')

sport_games = games[(games['Genre'] == 'Sports') & games['Critic_Score'].notna()]
rating_mean = sport_games['Critic_Score'].mean()
print(f'\nСреднее значение рейтинга критиков составляет {round(rating_mean, 1)}')

print('\nВыдвигаем следующие гипотезы:\n- нулевая гипотеза (H0): отношение критиков посредственное, если средний рейтинг <= 50\n- альтернативная гипотеза (H1): отношение критиков хорошее или превосходное, если средний рейтинг > 50')
print('Уровень значимости примем 0,05')

print('\nИспользуем односторонний t-тест Стьюдента для одной выборки.')
t_stat, p_value = st.ttest_1samp(sport_games['Critic_Score'], 50, alternative='greater')
print(f'\nРасчётное р-значение t-теста Стьюдента составляет {round(p_value, 3)}, что "превосходит" требуемый уровень значимости 0,05, это позволяет отклонить нулевую гипотезу и сделать вывод о наличии статистически значимого превышения среднего значения {round(rating_mean, 0)} относительно константы 50')

print('\nВыдвигаем следующие гипотезы:\n- нулевая гипотеза (H0): отношение критиков хорошее, если средний рейтинг <= 75\n- альтернативная гипотеза (H1): отношение критиков превосходное, если средний рейтинг > 75')
t_stat, p_value = st.ttest_1samp(sport_games['Critic_Score'], 75, alternative='greater')
print(f'\nРасчётное р-значение t-теста Стьюдента составляет {round(p_value, 3)}, что "не превосходит" требуемый уровень значимости 0,05, это не позволяет отклонить нулевую гипотезу, делаем вывод об отсутствии статистически значимого превышения среднего значения {round(rating_mean, 0)} относительно константы 75')

print('\nВывод: критики относятся к спортивным играм в среднем хорошо')


Как критики относятся к спортивным играм? Предположим, что среднему значению рейтинга до 50 соответсвует посредственное отношение, от 50 до 75 - хорошее, более 75 - превосходное.

Среднее значение рейтинга критиков составляет 72.0

Выдвигаем следующие гипотезы:
- нулевая гипотеза (H0): отношение критиков посредственное, если средний рейтинг <= 50
- альтернативная гипотеза (H1): отношение критиков хорошее или превосходное, если средний рейтинг > 50
Уровень значимости примем 0,05

Используем односторонний t-тест Стьюдента для одной выборки.

Расчётное р-значение t-теста Стьюдента составляет 0.0, что "превосходит" требуемый уровень значимости 0,05, это позволяет отклонить нулевую гипотезу и сделать вывод о наличии статистически значимого превышения среднего значения 72.0 относительно константы 50

Выдвигаем следующие гипотезы:
- нулевая гипотеза (H0): отношение критиков хорошее, если средний рейтинг <= 75
- альтернативная гипотеза (H1): отношение критиков превосходное, если средний рейтин

In [3]:
PC_games = games[(games['Platform'] == 'PC') & games['Critic_Score'].notna()]
PS4_games = games[(games['Platform'] == 'PS4') & games['Critic_Score'].notna()]
print('Критикам больше нравятся игры на PC или на PS4?')

PC_games_mean = PC_games['Critic_Score'].mean()
PS4_games_mean = PS4_games['Critic_Score'].mean()
print(f'\nСреднее значение рейтинга критиков:\n- для игр на PC {round(PC_games_mean, 1)}\n- для игр на PS4 {round(PS4_games_mean, 1)}')

print('\nВыдвигаем следующие гипотезы:\n- нулевая гипотеза (H0): критикам нравятся игры на PC или PS4 в среднем одинаково\n- альтернативная гипотеза (H1): критикам нравятся игры на PC или PS4 в разной мере')
print('Уровень значимости примем 0,05')

print('\nИспользуем двусторонний двухвыборочный t-тест Стьюдента.')
t_stat, p_value = st.ttest_ind(PC_games['Critic_Score'], PS4_games['Critic_Score'], equal_var=False)
print(f'\nРасчётное значение t-статистики составляет {round(t_stat, 2)}, положительное значение указывает на большую величину среднего значения рейтинга критиков для игр на PC по сравнению с играми на PS4. Расчётное р-значение составляет {round(p_value, 5)}, что "превосходит" требуемый уровень значимости 0,05, это позволяет отклонить нулевую гипотезу и сделать вывод о наличии статистически значимой разницы в средних оценках критиков для игр на PC и PS4')

print('\nВывод: критикам больше нравятся игры на PC по сравнению с PS4')

Критикам больше нравятся игры на PC или на PS4?

Среднее значение рейтинга критиков:
- для игр на PC 75.9
- для игр на PS4 72.1

Выдвигаем следующие гипотезы:
- нулевая гипотеза (H0): критикам нравятся игры на PC или PS4 в среднем одинаково
- альтернативная гипотеза (H1): критикам нравятся игры на PC или PS4 в разной мере
Уровень значимости примем 0,05

Используем двусторонний двухвыборочный t-тест Стьюдента.

Расчётное значение t-статистики составляет 4.31, положительное значение указывает на большую величину среднего значения рейтинга критиков для игр на PC по сравнению с играми на PS4. Расчётное р-значение составляет 2e-05, что "превосходит" требуемый уровень значимости 0,05, это позволяет отклонить нулевую гипотезу и сделать вывод о наличии статистически значимой разницы в средних оценках критиков для игр на PC и PS4

Вывод: критикам больше нравятся игры на PC по сравнению с PS4


In [4]:
shooter_games = games[(games['Genre'] == 'Shooter') & games['Critic_Score'].notna()]
strategy_games = games[(games['Genre'] == 'Strategy') & games['Critic_Score'].notna()]
print('Критикам больше нравятся стрелялки или стратегии?')

shooter_games_mean = shooter_games['Critic_Score'].mean()
strategy_games_mean = strategy_games['Critic_Score'].mean()
print(f'\nСреднее значение рейтинга критиков:\n- для стрелялок {round(shooter_games_mean, 1)}\n- для стратегий {round(strategy_games_mean, 1)}')

print('\nВыдвигаем следующие гипотезы:\n- нулевая гипотеза (H0): критикам нравятся стрелялки и стратегии в среднем одинаково\n- альтернативная гипотеза (H1): критикам нравятся стрелялки и стратегии в разной мере')
print('Уровень значимости примем 0,05')

print('\nИспользуем двусторонний двухвыборочный t-тест Стьюдента.')
t_stat, p_value = st.ttest_ind(shooter_games['Critic_Score'], strategy_games['Critic_Score'], equal_var=False)
print(f'\nРасчётное значение t-статистики составляет {round(t_stat, 2)}, отрицательное значение указывает на меньшую величину среднего значения рейтинга критиков для стрелялок по сравнению со стратегиями. Расчётное р-значение составляет {round(p_value, 3)}, что "превосходит" требуемый уровень значимости 0,05, это позволяет отклонить нулевую гипотезу и сделать вывод о наличии статистически значимой разницы в средних оценках критиков для стрелялок и стратегий')

print('\nВывод: критикам больше нравятся стратегии по сравнению со стрелялками')

Критикам больше нравятся стрелялки или стратегии?

Среднее значение рейтинга критиков:
- для стрелялок 70.2
- для стратегий 72.1

Выдвигаем следующие гипотезы:
- нулевая гипотеза (H0): критикам нравятся стрелялки и стратегии в среднем одинаково
- альтернативная гипотеза (H1): критикам нравятся стрелялки и стратегии в разной мере
Уровень значимости примем 0,05

Используем двусторонний двухвыборочный t-тест Стьюдента.

Расчётное значение t-статистики составляет -2.3, отрицательное значение указывает на меньшую величину среднего значения рейтинга критиков для стрелялок по сравнению со стратегиями. Расчётное р-значение составляет 0.022, что "превосходит" требуемый уровень значимости 0,05, это позволяет отклонить нулевую гипотезу и сделать вывод о наличии статистически значимой разницы в средних оценках критиков для стрелялок и стратегий

Вывод: критикам больше нравятся стратегии по сравнению со стрелялками


### Задание 2

In [118]:
# Загружаем файл с данными
messages = pd.read_csv('spam.csv')
# Приводим все слова к нижнему регистру
messages['Message'] = messages['Message'].str.lower()
# Удаляем мусорные символы
messages['Message'] = messages['Message'].apply(lambda text: re.sub('[\W_]+',' ', text))
# Разделим на слова
messages['Message'] = messages['Message'].apply(lambda text: text.split())
# Удалим стоп-слова
stopwords_set = set(stopwords.words('english'))
messages['Message'] = messages['Message'].apply(lambda text: [word for word in text if word not in stopwords_set])
# Приведём слова к простым формам
wordnet_lemmatizer = WordNetLemmatizer()
messages['Message'] = messages['Message'].apply(lambda text: [wordnet_lemmatizer.lemmatize(word) for word in text])
# Преобразуем отдельные слова обратно в сообщения
messages['Message'] = messages['Message'].apply(lambda text: ' '.join(text))
# Преобразуем сообщения в вектора TF-IDF
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(messages['Message'])
names = tfidf.get_feature_names()
tfidf_matrix = pd.DataFrame(tfidf_matrix.toarray(), columns=names)
# Разделим данные на тестовые и тренировочные в соотношении 30/70
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, messages['Category'], test_size = 0.3, random_state = 42)
# Обучим модель лигистической регрессии
logreg = LogisticRegression(random_state = 42)
logreg.fit(X_train, y_train)
# Запишем результаты модели в отдельный датафрейм
result = pd.DataFrame([y_test.tolist(), logreg.predict(X_test).tolist(), messages.iloc[y_test.index]['Message'].tolist()], index = ['Test', 'Predict', 'Message']).T
# Точность модели оценим отдельно для спама и не спама
ham_accuracy = len(result[(result['Test'] == 'ham') & (result['Predict'] == 'spam')]) / len(result[result['Test'] == 'ham'])
spam_accuracy = len(result[(result['Test'] == 'spam') & (result['Predict'] == 'ham')]) / len(result[result['Test'] == 'spam'])
print(f'Модель определяет спам с точностью {round(1 - spam_accuracy, 3) * 100}%, при этом определяет сообщения, не являющиеся спамом с точностью {round(1 - ham_accuracy, 3) * 100}%')
# Матрица ошибок
conf_matrix = confusion_matrix(y_test, logreg.predict(X_test))
print(f'\nИстино-положительных решений {conf_matrix[0, 0]}')
print(f'Истино-отрицательных решений {conf_matrix[1, 1]}')
print(f'Ложно-положительных решений {conf_matrix[0, 1]}')
print(f'Ложно-отрицательных решений {conf_matrix[1, 0]}')
# Датафрейм, который содержит все исходные тексты сообщений, классифицированные неправильно
error_result = result[((result['Test'] == 'ham') & (result['Predict'] == 'spam')) | ((result['Test'] == 'spam') & (result['Predict'] == 'ham'))]


Модель определяет спам с точностью 70.5%, при этом определяет сообщения, не являющиеся спамом с точностью 99.8%

Истино-положительных решений 1445
Истино-отрицательных решений 158
Ложно-положительных решений 3
Ложно-отрицательных решений 66
